### import necessary dependencies

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# initialize the spark session
spark = SparkSession.builder.appName('NugaBankETL').getOrCreate()

In [3]:
spark


### DATA  EXTRACTION

In [4]:
nuga_bank_df = spark.read.csv(r'dataset\nuga_bank_transactions.csv',header=True,inferSchema=True)

In [5]:
nuga_bank_df.show(5)

+--------------------+------+----------------+--------------+--------------------+------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+------+--------------+
|    Transaction_Date|Amount|Transaction_Type| Customer_Name|    Customer_Address|     Customer_City|Customer_State|    Customer_Country|             Company|           Job_Title|               Email|       Phone_Number|Credit_Card_Number|                IBAN|Currency_Code|Random_Number|Category|Group|Is_Active|        Last_Updated|         Description|Gender|Marital_Status|
+--------------------+------+----------------+--------------+--------------------+------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+-----

In [6]:
nuga_bank_df.printSchema()

root
 |-- Transaction_Date: timestamp (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Transaction_Type: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Customer_Address: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Customer_State: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Job_Title: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone_Number: string (nullable = true)
 |-- Credit_Card_Number: long (nullable = true)
 |-- IBAN: string (nullable = true)
 |-- Currency_Code: string (nullable = true)
 |-- Random_Number: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Group: string (nullable = true)
 |-- Is_Active: string (nullable = true)
 |-- Last_Updated: timestamp (nullable = true)
 |-- Description: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital_Status: string (nullable = true)

In [7]:
#Data cleaning and transformation
nuga_bank_df.columns


['Transaction_Date',
 'Amount',
 'Transaction_Type',
 'Customer_Name',
 'Customer_Address',
 'Customer_City',
 'Customer_State',
 'Customer_Country',
 'Company',
 'Job_Title',
 'Email',
 'Phone_Number',
 'Credit_Card_Number',
 'IBAN',
 'Currency_Code',
 'Random_Number',
 'Category',
 'Group',
 'Is_Active',
 'Last_Updated',
 'Description',
 'Gender',
 'Marital_Status']

In [8]:
# no of rows
num_rows = nuga_bank_df.count()
num_rows

1000000

In [9]:
# num of columns
num_columns = len(nuga_bank_df.columns)
num_columns

23

In [10]:
# check for null values
for column in nuga_bank_df.columns:
    print(column,'Nulls',nuga_bank_df.filter(nuga_bank_df[column].isNull()).count())

Transaction_Date Nulls 0
Amount Nulls 0
Transaction_Type Nulls 0
Customer_Name Nulls 100425
Customer_Address Nulls 100087
Customer_City Nulls 100034
Customer_State Nulls 100009
Customer_Country Nulls 100672
Company Nulls 100295
Job_Title Nulls 99924
Email Nulls 100043
Phone_Number Nulls 100524
Credit_Card_Number Nulls 100085
IBAN Nulls 100300
Currency_Code Nulls 99342
Random_Number Nulls 99913
Category Nulls 100332
Group Nulls 100209
Is_Active Nulls 100259
Last_Updated Nulls 100321
Description Nulls 100403
Gender Nulls 99767
Marital_Status Nulls 99904


In [12]:
# how to fill up missing value 
nuga_bank_df_clean = nuga_bank_df.fillna({
    'Customer_Name' : 'unknown',
    'Customer_Address' : 'unknown',
    'Customer_City' : 'unknown',
    'Customer_State' : 'unknown',
    'Customer_Country' : 'unknown',
    'Company' : 'unknown',
    'Job_Title' : 'unknown',
    'Email' : 'unknown',
    'Phone_Number' : 'unknown',
    'Credit_Card_Number' : 0,
    'IBAN' : 'unknown',
    'Currency_Code' : 'unknown',
    'Random_Number' : 0.0,
    'Category' : 'unknown',
    'Group' : 'unknown',
    'Is_Active' : 'unknown',
    'Description' : 'unknown',
    'Gender' : 'unknown',
    'Marital_Status' : 'unknown'
})



In [13]:
# check for null values
for column in nuga_bank_df_clean.columns:
    print(column,'Nulls',nuga_bank_df_clean.filter(nuga_bank_df_clean[column].isNull()).count())

Transaction_Date Nulls 0
Amount Nulls 0
Transaction_Type Nulls 0
Customer_Name Nulls 0
Customer_Address Nulls 0
Customer_City Nulls 0
Customer_State Nulls 0
Customer_Country Nulls 0
Company Nulls 0
Job_Title Nulls 0
Email Nulls 0
Phone_Number Nulls 0
Credit_Card_Number Nulls 0
IBAN Nulls 0
Currency_Code Nulls 0
Random_Number Nulls 0
Category Nulls 0
Group Nulls 0
Is_Active Nulls 0
Last_Updated Nulls 100321
Description Nulls 0
Gender Nulls 0
Marital_Status Nulls 0


In [14]:
nuga_bank_df_clean = nuga_bank_df_clean.na.drop(subset=['Last_Updated'])

In [15]:
num_rows = nuga_bank_df_clean.count()
num_rows

899679

In [16]:
# to view the summary statistics of the data
nuga_bank_df_clean.describe().show()

+-------+------------------+----------------+-------------+--------------------+-------------+--------------+----------------+------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------+-----------------+--------+-------+---------+--------------------+-------+--------------+
|summary|            Amount|Transaction_Type|Customer_Name|    Customer_Address|Customer_City|Customer_State|Customer_Country|     Company|         Job_Title|              Email|        Phone_Number|  Credit_Card_Number|                IBAN|Currency_Code|    Random_Number|Category|  Group|Is_Active|         Description| Gender|Marital_Status|
+-------+------------------+----------------+-------------+--------------------+-------------+--------------+----------------+------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------+-----------------+--------+-------+---------+----------

In [17]:
nuga_bank_df_clean.columns

['Transaction_Date',
 'Amount',
 'Transaction_Type',
 'Customer_Name',
 'Customer_Address',
 'Customer_City',
 'Customer_State',
 'Customer_Country',
 'Company',
 'Job_Title',
 'Email',
 'Phone_Number',
 'Credit_Card_Number',
 'IBAN',
 'Currency_Code',
 'Random_Number',
 'Category',
 'Group',
 'Is_Active',
 'Last_Updated',
 'Description',
 'Gender',
 'Marital_Status']

In [18]:
# trransaction table
transaction =nuga_bank_df_clean.select('Transaction_Date',
'Amount',
 'Transaction_Type')

In [19]:
transaction.show()

+--------------------+------+----------------+
|    Transaction_Date|Amount|Transaction_Type|
+--------------------+------+----------------+
|2024-03-23 15:38:...| 34.76|      Withdrawal|
|2024-04-22 19:15:...|163.92|      Withdrawal|
|2024-04-12 19:46:...|386.32|      Withdrawal|
|2024-04-17 15:29:...|407.15|         Deposit|
|2024-02-10 01:51:...|161.31|         Deposit|
|2024-02-10 22:56:...|764.34|        Transfer|
|2024-04-07 00:07:...|734.59|         Deposit|
|2024-03-08 01:51:...|592.43|         Deposit|
|2024-02-01 12:34:...| 927.1|         Deposit|
|2024-03-22 16:46:...| 66.59|        Transfer|
|2024-04-23 13:30:...| 246.3|      Withdrawal|
|2024-01-13 01:22:...|782.32|      Withdrawal|
|2024-02-25 15:16:...|818.42|      Withdrawal|
|2024-01-01 20:55:...|352.23|      Withdrawal|
|2024-01-19 00:01:...|316.19|      Withdrawal|
|2024-04-09 14:40:...|662.26|      Withdrawal|
|2024-04-15 04:58:...|893.73|         Deposit|
|2024-04-12 14:32:...|746.22|      Withdrawal|
|2024-02-26 1

In [20]:
## to add the transaction_id column
transaction = transaction.withColumn('transaction_id',monotonically_increasing_id())

In [21]:
transaction.show()

+--------------------+------+----------------+--------------+
|    Transaction_Date|Amount|Transaction_Type|transaction_id|
+--------------------+------+----------------+--------------+
|2024-03-23 15:38:...| 34.76|      Withdrawal|             0|
|2024-04-22 19:15:...|163.92|      Withdrawal|             1|
|2024-04-12 19:46:...|386.32|      Withdrawal|             2|
|2024-04-17 15:29:...|407.15|         Deposit|             3|
|2024-02-10 01:51:...|161.31|         Deposit|             4|
|2024-02-10 22:56:...|764.34|        Transfer|             5|
|2024-04-07 00:07:...|734.59|         Deposit|             6|
|2024-03-08 01:51:...|592.43|         Deposit|             7|
|2024-02-01 12:34:...| 927.1|         Deposit|             8|
|2024-03-22 16:46:...| 66.59|        Transfer|             9|
|2024-04-23 13:30:...| 246.3|      Withdrawal|            10|
|2024-01-13 01:22:...|782.32|      Withdrawal|            11|
|2024-02-25 15:16:...|818.42|      Withdrawal|            12|
|2024-01

In [22]:
# to rearrange transaction
transaction = transaction.select('transaction_id','Transaction_Date',
 'Amount','Transaction_Type')

In [23]:
transaction.show()

+--------------+--------------------+------+----------------+
|transaction_id|    Transaction_Date|Amount|Transaction_Type|
+--------------+--------------------+------+----------------+
|             0|2024-03-23 15:38:...| 34.76|      Withdrawal|
|             1|2024-04-22 19:15:...|163.92|      Withdrawal|
|             2|2024-04-12 19:46:...|386.32|      Withdrawal|
|             3|2024-04-17 15:29:...|407.15|         Deposit|
|             4|2024-02-10 01:51:...|161.31|         Deposit|
|             5|2024-02-10 22:56:...|764.34|        Transfer|
|             6|2024-04-07 00:07:...|734.59|         Deposit|
|             7|2024-03-08 01:51:...|592.43|         Deposit|
|             8|2024-02-01 12:34:...| 927.1|         Deposit|
|             9|2024-03-22 16:46:...| 66.59|        Transfer|
|            10|2024-04-23 13:30:...| 246.3|      Withdrawal|
|            11|2024-01-13 01:22:...|782.32|      Withdrawal|
|            12|2024-02-25 15:16:...|818.42|      Withdrawal|
|       

In [24]:
customer =nuga_bank_df_clean.select('Customer_Name','Customer_Address','Customer_City','Customer_State','Customer_Country', 'Email',
 'Phone_Number')

In [25]:
customer.show()

+-----------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------------+
|    Customer_Name|    Customer_Address|       Customer_City|Customer_State|    Customer_Country|               Email|       Phone_Number|
+-----------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------------+
|       James Neal|54912 Holmes Lodg...|   West Keithborough|       Florida|                Togo|             unknown|  493.720.6609x7545|
|      Thomas Long| 1133 Collin Passage|          Joshuabury|   Connecticut|Lao People's Demo...|michellelynch@exa...|      (497)554-3317|
|   Ashley Shelton|5297 Johnson Port...|         North Maria|    New Jersey|              Bhutan| ljordan@example.org|      (534)769-3072|
|    James Rosario|56955 Moore Glens...|  North Michellefurt|    New Mexico|             Iceland|parkerjames@examp...|+1-447-900-1320x257|
|   Miguel Leonard|262 Beck

In [28]:
# to add column 
customer = customer.withColumn('customer_id',monotonically_increasing_id())

In [31]:
customer.show()

+-----------+-----------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------------+
|customer_id|    Customer_Name|    Customer_Address|       Customer_City|Customer_State|    Customer_Country|               Email|       Phone_Number|
+-----------+-----------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------------+
|          0|       James Neal|54912 Holmes Lodg...|   West Keithborough|       Florida|                Togo|             unknown|  493.720.6609x7545|
|          1|      Thomas Long| 1133 Collin Passage|          Joshuabury|   Connecticut|Lao People's Demo...|michellelynch@exa...|      (497)554-3317|
|          2|   Ashley Shelton|5297 Johnson Port...|         North Maria|    New Jersey|              Bhutan| ljordan@example.org|      (534)769-3072|
|          3|    James Rosario|56955 Moore Glens...|  North Michellefurt|    New Mexico|      

In [30]:
# to rearrange customer 
customer = customer.select('customer_id','Customer_Name','Customer_Address','Customer_City','Customer_State','Customer_Country', 'Email',
 'Phone_Number') 

In [32]:
employee =nuga_bank_df_clean.select('Company','Job_Title', 'Gender','Marital_Status')

In [33]:
employee.show()

+--------------------+--------------------+-------+--------------+
|             Company|           Job_Title| Gender|Marital_Status|
+--------------------+--------------------+-------+--------------+
|Benson, Johnson a...|             unknown|  Other|      Divorced|
|             unknown|   Food technologist| Female|       Married|
|       Jones-Mueller|Database administ...|  Other|       unknown|
|       Vargas-Harris|Horticultural the...|unknown|       unknown|
|Richardson, Gonza...|   Minerals surveyor| Female|       Married|
|           Smith Ltd| Seismic interpreter|  Other|       Married|
|         Wade-Kelley|  Surveyor, minerals|   Male|       unknown|
|             unknown|Medical laborator...| Female|        Single|
|         Lindsey LLC|Programmer, appli...| Female|        Single|
|         Carroll LLC|             unknown|   Male|        Single|
|             unknown|     Sales executive|   Male|      Divorced|
|Lynch, Hayes and ...|       Stage manager|unknown|       Marr

In [34]:
# to add column
employee = employee.withColumn('employee_id',monotonically_increasing_id())

In [35]:
employee.show()

+--------------------+--------------------+-------+--------------+-----------+
|             Company|           Job_Title| Gender|Marital_Status|employee_id|
+--------------------+--------------------+-------+--------------+-----------+
|Benson, Johnson a...|             unknown|  Other|      Divorced|          0|
|             unknown|   Food technologist| Female|       Married|          1|
|       Jones-Mueller|Database administ...|  Other|       unknown|          2|
|       Vargas-Harris|Horticultural the...|unknown|       unknown|          3|
|Richardson, Gonza...|   Minerals surveyor| Female|       Married|          4|
|           Smith Ltd| Seismic interpreter|  Other|       Married|          5|
|         Wade-Kelley|  Surveyor, minerals|   Male|       unknown|          6|
|             unknown|Medical laborator...| Female|        Single|          7|
|         Lindsey LLC|Programmer, appli...| Female|        Single|          8|
|         Carroll LLC|             unknown|   Male| 

In [36]:
# to rearrange employee
employee = employee.select('employee_id','Company','Job_Title', 'Gender','Marital_Status')

In [37]:
employee.show()

+-----------+--------------------+--------------------+-------+--------------+
|employee_id|             Company|           Job_Title| Gender|Marital_Status|
+-----------+--------------------+--------------------+-------+--------------+
|          0|Benson, Johnson a...|             unknown|  Other|      Divorced|
|          1|             unknown|   Food technologist| Female|       Married|
|          2|       Jones-Mueller|Database administ...|  Other|       unknown|
|          3|       Vargas-Harris|Horticultural the...|unknown|       unknown|
|          4|Richardson, Gonza...|   Minerals surveyor| Female|       Married|
|          5|           Smith Ltd| Seismic interpreter|  Other|       Married|
|          6|         Wade-Kelley|  Surveyor, minerals|   Male|       unknown|
|          7|             unknown|Medical laborator...| Female|        Single|
|          8|         Lindsey LLC|Programmer, appli...| Female|        Single|
|          9|         Carroll LLC|             unkno

In [41]:
fact_table = nuga_bank_df_clean.join(customer,['Customer_Name','Customer_Address','Customer_City','Customer_State','Customer_Country', 'Email',
 'Phone_Number'],'left')\
    .join(transaction,['Transaction_Date','Amount','Transaction_Type'],'left')\
    .join(employee,['Company','Job_Title', 'Gender','Marital_Status'],'left')\
    .select('transaction_id','customer_id','employee_id','Credit_Card_Number','IBAN', 'Currency_Code','Random_Number',\
            'Category','Group','Is_Active','Last_Updated','Description',)
 

In [42]:
fact_table.show()

+--------------+-----------+-----------+-------------------+--------------------+-------------+-------------+--------+-------+---------+--------------------+--------------------+
|transaction_id|customer_id|employee_id| Credit_Card_Number|                IBAN|Currency_Code|Random_Number|Category|  Group|Is_Active|        Last_Updated|         Description|
+--------------+-----------+-----------+-------------------+--------------------+-------------+-------------+--------+-------+---------+--------------------+--------------------+
|   17179869198|17179869198|17179869198|    180067592769732|GB92JVMY004197871...|          EGP|       7198.0|       A|      Z|      Yes|2023-10-12 22:25:...|Before story prof...|
|            18|         18|         18|    213112163828334|GB50TJFN039979307...|          SVC|       7382.0|       B|      Z|      Yes|2020-01-19 18:19:...|Great evening so ...|
|   17179869199|17179869199|17179869199|   4239162655922295|GB96MEEY268453596...|          BTN|       560

In [ ]:
#outputting the transformed data to parquet file
transaction.write.mode('overwrite').parquet(r'dataset/transaction')
employee.write.mode('overwrite').parquet(r'dataset/employee')
customer.write.mode('overwrite').parquet(r'dataset/customer')
fact_table.write.mode('overwrite').parquet(r'dataset/fact_table')
